# Optimizing `get_pixel_paths`

Sean

Here we show how the method to get the hive paths of the partition files was improved

Before:

In [5]:
from hipscat.pixel_math import HealpixPixel
import hipscat as hc
from hipscat.io.file_io.file_pointer import get_fs

In [8]:
base_dir = "/test/base_dir"
pixels = [HealpixPixel(0, 0), HealpixPixel(0, 1), HealpixPixel(3, 100)]

paths = [
    hc.io.paths.pixel_catalog_file(
        catalog_base_dir=base_dir,
        pixel_order=pixel.order,
        pixel_number=pixel.pixel,
    )
    for pixel in pixels
]
paths

['/test/base_dir/Norder=0/Dir=0/Npix=0.parquet',
 '/test/base_dir/Norder=0/Dir=0/Npix=1.parquet',
 '/test/base_dir/Norder=3/Dir=0/Npix=100.parquet']

`pixel_catalog_files` computes pixel_dir and calls os.path.join

Profiling shows it's slow:

![thing](images/Screenshot%202024-04-04%20at%202.36.36%E2%80%AFPM.png)

New method:

In [6]:
from hipscat.io.paths import ORDER_DIRECTORY_PREFIX, DIR_DIRECTORY_PREFIX, PIXEL_DIRECTORY_PREFIX


def pixel_catalog_files(catalog_base_dir, pixels, storage_options):
    fs, _ = get_fs(catalog_base_dir, storage_options)
    base_path_stripped = catalog_base_dir.removesuffix(fs.sep)
    return [fs.sep.join([base_path_stripped,
                         f"{ORDER_DIRECTORY_PREFIX}={pixel.order}",
                         f"{DIR_DIRECTORY_PREFIX}={pixel.pixel // 10000 * 10000}",
                         f"{PIXEL_DIRECTORY_PREFIX}={pixel.pixel}.parquet"
                         ]) for pixel in pixels]

In [9]:
pixel_catalog_files(base_dir, pixels, None)

['/test/base_dir/Norder=0/Dir=0/Npix=0.parquet',
 '/test/base_dir/Norder=0/Dir=0/Npix=1.parquet',
 '/test/base_dir/Norder=3/Dir=0/Npix=100.parquet']

new profiling:

![thing](images/Screenshot%202024-04-04%20at%203.10.22%E2%80%AFPM.png)